In [139]:
import pandas as pd#read in data using pandas

In [140]:
df = pd.read_csv('data_new.csv')#check data has been read in properly
df.head()

,Unnamed: 0,Amps,Volts,Speed (mm/s),Heat Input (kJ/mm),Weld_Quality
0,0,76.866867,19.135135,1.726537,2.335255,20.0
1,1,131.961962,15.693694,1.852943,1.749069,30.0
2,2,144.616617,27.531532,2.586757,2.282402,30.0
3,3,74.370370,23.369369,2.396326,0.754474,20.0
4,4,68.516517,17.531532,2.013824,2.114234,20.0


In [141]:
#create a dataframe with all training data except the target column
X = df.drop(columns=['Weld_Quality',"Unnamed: 0"])

#check that the target variable has been removed
X.head()

,Amps,Volts,Speed (mm/s),Heat Input (kJ/mm)
0,76.866867,19.135135,1.726537,2.335255
1,131.961962,15.693694,1.852943,1.749069
2,144.616617,27.531532,2.586757,2.282402
3,74.370370,23.369369,2.396326,0.754474
4,68.516517,17.531532,2.013824,2.114234


In [142]:
len(X)

1000

In [143]:
y = df[['Weld_Quality']]

In [144]:
y=y.replace([10, 20 ,30], ['Lack of Fusion', 'Good Weld', 'Burn Through'])

In [145]:
y.head()

,Weld_Quality
0,Good Weld
1,Burn Through
2,Burn Through
3,Good Weld
4,Good Weld


In [146]:
len(y)

1000

In [147]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.utils import np_utils

In [148]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [149]:
y.head()

,Weld_Quality
0,Good Weld
1,Burn Through
2,Burn Through
3,Good Weld
4,Good Weld


In [150]:
dummy_y[2]

array([1., 0., 0.], dtype=float32)

In [151]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(X,encoded_y,test_size=.35,random_state=8)

In [152]:
from keras.models import Sequential
from keras.layers import Dense, Dropout #create model
model = Sequential()

In [153]:
#get number of columns in training data
n_cols = train_X.shape[1]

#add model layers
model.add(Dense(6, activation='relu', input_shape=(n_cols,)))
model.add(Dense(4,activation="softmax"))
model.add(Dense(1, activation='softmax'))




model.compile(optimizer='adam',loss="mean_squared_error")

In [154]:
from keras.callbacks import EarlyStopping #set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)#train model
model.fit(train_X, train_y, validation_split=0.4, epochs=25, callbacks=[early_stopping_monitor])

Train on 390 samples, validate on 260 samples
Epoch 1/25
390/390 [==============================] - 3s 8ms/step - loss: 0.7641 - val_loss: 0.7385
Epoch 2/25
390/390 [==============================] - 0s 241us/step - loss: 0.7641 - val_loss: 0.7385
Epoch 3/25
390/390 [==============================] - 0s 223us/step - loss: 0.7641 - val_loss: 0.7385
Epoch 4/25
390/390 [==============================] - 0s 249us/step - loss: 0.7641 - val_loss: 0.7385


In [155]:
test_y_predictions = model.predict(test_X)


In [156]:
accuracy = model.evaluate(test_X, test_y)
# Print accuracy
print('Accuracy:', accuracy)

350/350 [==============================] - 0s 78us/step
Accuracy: 0.7428571418353489


In [157]:
test_X

,Amps,Volts,Speed (mm/s),Heat Input (kJ/mm)
908,151.417417,15.855856,2.647497,1.785105
265,72.820821,28.810811,2.749279,2.171892
627,122.578579,27.891892,1.555806,2.921441
327,115.175175,18.684685,2.693463,2.409730
244,105.877878,27.819820,2.329019,1.513634
...,...,...,...,...
874,152.192192,25.189189,1.874284,1.090811
521,89.005005,25.531532,2.439009,2.815736
935,108.632633,23.495495,1.944875,1.773093
696,123.353353,13.369369,2.320811,2.983904


In [138]:
model.predict((np.array([[153.6,15.7,2.67,3.23]])))

array([[1.]], dtype=float32)